In [102]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [39]:
url_base='https://www.hch.gov.tw'
url_work='/?aid=506&pid=0&page_name=list&pageNo=1'
url_full=url_base+url_work_table
g=requests.get(url_full)
soup=BeautifulSoup(g.content, 'html.parser')

# Get pages link

In [42]:
pages_link=[]
for link in soup.find('ul',class_="pagination pagination-sm justify-content-center"):
    if link.find("a"):
        if link.find("a") == -1 or link.text == "第一頁" or link.text == "«" or link.text == "»" or link.text == "最末頁" or link.text == " ":
            pass
        elif link.find('a').has_attr("disabled"):
            #print('skip:',link.text)
            pages_link.append(url_full)
        else:
            no_page_link=url_full.replace("pageNo=1","pageNo=")
            pages_link.append(no_page_link+link.find('a').text)
pages_link

['https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=1',
 'https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=2',
 'https://www.hch.gov.tw/?aid=506&pid=0&page_name=list&pageNo=3']

# Get wrok table

first page

In [96]:
def get_each_page_wrok_table(soup,work_table):
    for work in soup.find_all('table',class_="table table-striped table-hover news_table")[0].find("tbody").find_all("tr"):
        all_td = work.find_all("td")
        title = all_td[3].text
        place = all_td[0].text
        deadline = all_td[4].text
        originization = all_td[2].text
        link={}
        for i in all_td[5].find_all("a"):
            link[i.text]=url_base+i.get('href')
        print("召聘職稱",title,'院區',place,"截止日期",deadline,"職缺單位",originization,"links",link)
        work_table.append([title, deadline, originization, place, link ])
    return work_table

In [90]:
def get_base_web_data(url_base,url_work_table=None):
    if url_work_table != None:
        url = url_base + url_work_table
    else:
        url = url_base
    g=requests.get(url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [97]:
work_table = []
for page in pages_link:
    soup = get_base_web_data(page)
    work_table = get_each_page_wrok_table(soup,work_table)

召聘職稱 院聘醫事檢驗師 院區 竹北院區 截止日期 2022-05-16 職缺單位 醫療單位檢驗醫學部 links {' 新竹臺大分院簡則-院聘醫事檢驗師': 'https://www.hch.gov.tw/public/jobnews/16517443876072.pdf', ' 1.信封封面': 'https://www.hch.gov.tw/public/jobnews/1651744387617.pdf', ' 2.台大醫院新竹臺大分院各類人員甄選登記表及查核授權書': 'https://www.hch.gov.tw/public/jobnews/16517443876257.pdf', ' 3.作業基金現職人員申請院內外補同意書(新竹臺大分院)': 'https://www.hch.gov.tw/public/jobnews/16517443876352.pdf'}
召聘職稱 院聘部分工時人員 院區 新竹醫院 截止日期 2022-05-31 職缺單位 醫療單位家庭醫學部 links {' 簡則-部份工時人員': 'https://www.hch.gov.tw/public/jobnews/16517303872988.pdf', ' 信封封面': 'https://www.hch.gov.tw/public/jobnews/16517303873153.pdf', ' 台大醫院新竹臺大分院各類人員甄選登記表及查核授權書': 'https://www.hch.gov.tw/public/jobnews/16517303873263.pdf', ' 作業基金現職人員申請院內外補同意書': 'https://www.hch.gov.tw/public/jobnews/16517303873428.pdf'}
召聘職稱 院聘助理管理師 院區 新竹醫院 截止日期 2022-05-15 職缺單位 醫療單位檢驗醫學部 links {' 簡則-院聘助理管理師': 'https://www.hch.gov.tw/public/jobnews/16517299573322.pdf', ' 信封封面': 'https://www.hch.gov.tw/public/jobnews/16517299573498.pdf', ' 台大醫院新竹臺大分院各類人員甄選登記表及查核授權書'

In [101]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" , '院區','連結'])

print(work_table)

                   召聘職稱          期限                職缺單位            院區  \
0               院聘醫事檢驗師  2022-05-16           醫療單位檢驗醫學部          竹北院區   
1              院聘部分工時人員  2022-05-31           醫療單位家庭醫學部          新竹醫院   
2               院聘助理管理師  2022-05-15           醫療單位檢驗醫學部          新竹醫院   
3                 院聘管理師  2022-05-27         醫療支援單位安全衛生室          竹北院區   
4               院聘醫事檢驗師  2022-05-15             醫療單位病理部          新竹醫院   
5                院聘主治醫師  2022-05-15         醫療單位外科部胸腔外科  新竹醫院竹北院區竹東院區   
6               院聘門診護理師  2022-06-30           醫療支援單位護理部          竹東院區   
7      院聘護理師(婦幼中心院聘護理師)  2022-08-01           醫療支援單位護理部          竹北院區   
8      院聘護理師(安寧共同照護護理師)  2022-08-01           醫療支援單位護理部          竹北院區   
9         工時護理師(病房/開刀房)  2022-08-01           醫療支援單位護理部          竹北院區   
10                院聘護理師  2022-08-01           醫療支援單位護理部          竹北院區   
11              院聘門診護理師  2022-08-01           醫療支援單位護理部          竹北院區   
12             院聘事務員(二)  2022-05-13    醫療支援單位醫療事務室櫃